```
openai==1.3.6
```

In [1]:
import json
import os
import pathlib

import pandas as pd
from api_logger import ChatCompletionHttpLogger
from api_logger import create_http_client
from openai import OpenAI

In [2]:
http_client = create_http_client(logger=ChatCompletionHttpLogger('logs'))
client = OpenAI(api_key=os.environ.get('OPENAI_API_KEY'), http_client=http_client)

In [3]:
def dispatch(message: str) -> str:
    response = client.chat.completions.create(model='gpt-4', messages=[{'role': 'user', 'content': message}])
    return response.choices[0].message.content

In [4]:
prompts = [
    'Write a python program to sum the list of numbers',
    'Who is the creator of Doom?',
    'What is the capital of the United States?',
    'Explain the process of photosynthesis in plants.',
    'What is the significance of the Turing Test in Artificial Intelligence?',
]
for prompt in prompts:
    dispatch(prompt)

In [5]:
logs = []
for file in pathlib.Path('logs').iterdir():
    with open(file) as _file:
        logs.append(json.load(_file))

In [6]:
# From GPT-4
def flatten_json(json_object, parent_key='', sep='.'):
    items = []
    if isinstance(json_object, dict):
        for key, value in json_object.items():
            new_key = f'{parent_key}{sep}{key}' if parent_key else key
            if isinstance(value, (dict, list)):
                items.extend(flatten_json(value, new_key, sep=sep).items())
            else:
                items.append((new_key, value))
    elif isinstance(json_object, list):
        for i, value in enumerate(json_object):
            new_key = f'{parent_key}{sep}{i}' if parent_key else str(i)
            if isinstance(value, (dict, list)):
                items.extend(flatten_json(value, new_key, sep=sep).items())
            else:
                items.append((new_key, value))
    else:
        items.append((parent_key, json_object))

    return dict(items)

In [7]:
_logs = []
for _map in logs:
    _logs.append(flatten_json(_map))

In [8]:
df = pd.DataFrame(_logs)
df

,status_code,url,start_time,end_time,latency_ms,request_body.messages.0.role,request_body.messages.0.content,request_body.model,response_body.id,response_body.object,...,response_body.choices.0.message.role,response_body.choices.0.message.content,response_body.choices.0.finish_reason,response_body.usage.prompt_tokens,response_body.usage.completion_tokens,response_body.usage.total_tokens,response_body.system_fingerprint,prompt_token_cost,completion_token_cost,total_token_cost
0,200,https://api.openai.com/v1/chat/completions,1.701786e+09,1.701786e+09,9206.016064,user,What is the significance of the Turing Test in...,gpt-4,chatcmpl-8SQfP3wSISatmgGj1J5jPUqMvNSVp,chat.completion,...,assistant,"The Turing Test, proposed by British mathemati...",stop,19,144,163,None,0.00057,0.00864,0.00921
1,200,https://api.openai.com/v1/chat/completions,1.701786e+09,1.701786e+09,15765.666008,user,Explain the process of photosynthesis in plants.,gpt-4,chatcmpl-8SQf94RzZ4G64yzIPEF7EOM3e5zyJ,chat.completion,...,assistant,"Photosynthesis is the process by which plants,...",stop,17,302,319,None,0.00051,0.01812,0.01863
2,200,https://api.openai.com/v1/chat/completions,1.701786e+09,1.701786e+09,1113.662243,user,What is the capital of the United States?,gpt-4,chatcmpl-8SQf85ZDiTuWCIh2venpkweSHtR8g,chat.completion,...,assistant,The capital of the United States is Washington...,stop,16,11,27,None,0.00048,0.00066,0.00114
3,200,https://api.openai.com/v1/chat/completions,1.701786e+09,1.701786e+09,1625.068188,user,Who is the creator of Doom?,gpt-4,chatcmpl-8SQf6MMMGrDOrRH1kM613nLrxeS9c,chat.completion,...,assistant,"Doom was created by a team at id Software, led...",stop,14,20,34,None,0.00042,0.00120,0.00162
4,200,https://api.openai.com/v1/chat/completions,1.701786e+09,1.701786e+09,10978.924990,user,Write a python program to sum the list of numbers,gpt-4,chatcmpl-8SQeyEfKa5f3kWOJIUYiVcSVztATU,chat.completion,...,assistant,"Sure, here is a simple Python program to sum t...",stop,17,136,153,None,0.00051,0.00816,0.00867


In [9]:
df.to_csv('logs.csv')